In [25]:
include("./prepare_data.jl")
include("./model.jl")

import .Model
import Flux

In [26]:
model = Model.model

Chain(
  Recur(
    LSTMCell(128 => 64),                # 49_536 parameters
  ),
  Recur(
    LSTMCell(64 => 32),                 # 12_480 parameters
  ),
  Flux.flatten,
  Dense(96 => 3),                       # 291 parameters
  NNlib.softmax,
)         # Total: 12 trainable arrays, 62_307 parameters,
          # plus 4 non-trainable, 192 parameters, summarysize 244.184 KiB.

In [28]:
x = rand(Float32, (128,3,4))
model(x)

3×4 Matrix{Float32}:
 0.287264  0.287657  0.284707  0.287036
 0.332946  0.321546  0.318121  0.308019
 0.37979   0.390797  0.397172  0.404945